In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('hand_landmarks.csv')

# Assuming the first column 'label' is binary and the rest are features
X = df.drop('label', axis=1).values
y = df['label'].values

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [2]:
# Define the model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # For binary classification
              metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                2048      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2593 (10.13 KB)
Trainable params: 2593 (10.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Train the model
model.fit(X_train, y_train, epochs=500, validation_split=0.1, batch_size=64, class_weight=class_weight_dict)

# Save the model
model.save('hand_model.h5')

joblib.dump(scaler, 'scaler.pkl')


Epoch 1/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0759 - accuracy: 0.9766 - val_loss: 0.0308 - val_accuracy: 0.9877
Epoch 2/500
12/12 [==============================] - 0s 5ms/step - loss: 0.0852 - accuracy: 0.9739 - val_loss: 0.0344 - val_accuracy: 0.9877
Epoch 3/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0788 - accuracy: 0.9766 - val_loss: 0.0353 - val_accuracy: 0.9753
Epoch 4/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0985 - accuracy: 0.9780 - val_loss: 0.0902 - val_accuracy: 0.9753
Epoch 5/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0956 - accuracy: 0.9835 - val_loss: 0.0426 - val_accuracy: 0.9877
Epoch 6/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0694 - accuracy: 0.9766 - val_loss: 0.0287 - val_accuracy: 0.9753
Epoch 7/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0873 - accuracy: 0.9725 - val_loss: 0.0269 - val_accuracy: 0.9877
Epoch 8/500
1

c:\Users\zachu\.conda\envs\hand_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

# Predict probabilities
probabilities = model.predict(X_test)

# Define a custom threshold
threshold = 0.99  # Adjust based on your requirement

# Apply the threshold
predictions = (probabilities > threshold).astype(int)

# Evaluate accuracy
test_accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy with threshold {threshold}:', test_accuracy)


7/7 [==============================] - 0s 1ms/step
Test Accuracy with threshold 0.99: 0.9802955665024631
